Naive Bayes Model 1 Healt data


Check if someone is obese or not

In [38]:
import pandas as pd
from pandas.api.types import CategoricalDtype

url = "https://raw.githubusercontent.com/Patrick0481/Data-mining-2025-course/refs/heads/main/Final_data.csv?token=GHSAT0AAAAAADNERMYHZFCTLIULVE5F4SY22HSINQQ"
rawDF = pd.read_csv(url)
rawDF.head()

HTTPError: HTTP Error 404: Not Found

In [ ]:
# use the dataframe loaded in cell 1
df = rawDF.copy()

def classify_bmi(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif bmi < 25:
        return 'Normal'
    elif bmi < 30:
        return 'Overweight'
    else:
        return 'Obese'

df['Obesity_Class'] = df['BMI'].apply(classify_bmi)
print(df[['BMI', 'Obesity_Class']].head()) #check if the code works

     BMI Obesity_Class
0  24.87        Normal
1  23.48        Normal
2  21.15        Normal
3  32.45         Obese
4  14.83   Underweight


In [34]:
#manually select features to use for classification
# use the actual column names from df 
selected_features = [
    'Age', 'Gender', 'Weight (kg)', 'Height (m)', 'BMI', 'Fat_Percentage',
    'Workout_Frequency (days/week)', 'Calories_Burned', 'diet_type',
    'Daily meals frequency', 'Water_Intake (liters)', 'Physical exercise',
    'Carbs', 'Proteins', 'Fats', 'cal_balance'
]

X = df[selected_features]
y = df['Obesity_Class']

In [18]:
#ussing SelectKBest to select top features
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder

# Target encoden als numeriek
y_encoded = LabelEncoder().fit_transform(df['Obesity_Class'])

# Selectie van numerieke kolommen
numeric_cols = df.select_dtypes(include=['int64','float64']).columns
X = df[numeric_cols].dropna()

# selecting top 10 features
selector = SelectKBest(score_func=f_classif, k=10)
X_new = selector.fit_transform(X, y_encoded)

selected_features = X.columns[selector.get_support()]
print("Top features:", selected_features)


Top features: Index(['Weight (kg)', 'Fat_Percentage', 'Water_Intake (liters)', 'BMI',
       'Calories', 'BMI_calc', 'protein_per_kg', 'cal_balance', 'lean_mass_kg',
       'Burns Calories (per 30 min)_bc'],
      dtype='object')


In [ ]:
#use RandomForest to determine feature importance
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

X = df.drop(columns=['Obesity_Class'])
y = df['Obesity_Class']

# Encodeer categorische waarden
X = pd.get_dummies(X, drop_first=True)

rf = RandomForestClassifier(random_state=42)
rf.fit(X, y)

importances = pd.Series(rf.feature_importances_, index=X.columns)
top_features = importances.sort_values(ascending=False).head(15)
print(top_features)

BMI                               0.236064
BMI_calc                          0.227400
Fat_Percentage                    0.078895
Weight (kg)                       0.065654
Calories                          0.044841
lean_mass_kg                      0.040301
protein_per_kg                    0.031420
Height (m)                        0.028208
cal_balance                       0.021509
Water_Intake (liters)             0.010496
Session_Duration (hours)          0.009815
expected_burn                     0.008953
Burns Calories (per 30 min)       0.008811
Burns Calories (per 30 min)_bc    0.008734
Carbs                             0.008717
dtype: float64


In [ ]:
from sklearn.feature_selection import mutual_info_classif
import numpy as np

X = pd.get_dummies(df[selected_features], drop_first=True) # Zorg ervoor dat we dezelfde features gebruiken
y = LabelEncoder().fit_transform(df['Obesity_Class']) 

bmi = mutual_info_classif(X, y) 
bmi_scores = pd.Series(mi, index=X.columns).sort_values(ascending=False) # Sorteer op belangrijkheid

print("Feature importance op basis van mutual information:")
print(bmi_scores)
#print(selected_features)

Feature importance op basis van mutual information:
BMI                               1.334439
BMI_calc                          1.334329
Fat_Percentage                    0.730535
Weight (kg)                       0.601831
Calories                          0.503457
lean_mass_kg                      0.422847
protein_per_kg                    0.348872
cal_balance                       0.250718
Burns Calories (per 30 min)_bc    0.151972
Water_Intake (liters)             0.105123
dtype: float64
Index(['Weight (kg)', 'Fat_Percentage', 'Water_Intake (liters)', 'BMI',
       'Calories', 'BMI_calc', 'protein_per_kg', 'cal_balance', 'lean_mass_kg',
       'Burns Calories (per 30 min)_bc'],
      dtype='object')


In [ ]:
#%pip install imbalanced-learn

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import pandas as pd

# --- 1. Selecteer features en target ---
X = df[selected_features].copy()
y = df['Obesity_Class']

# --- 2. Encodeer categorische kolommen automatisch ---
for col in X.select_dtypes(include=['object', 'category']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

# --- 3. Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- 4. Balans tussen klassen herstellen ---
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# --- 5. Normaliseer features (voor GaussianNB) ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)

# --- 6. Train het model ---
model = GaussianNB()
model.fit(X_train_scaled, y_train_res)

# --- 7. Evaluatie ---
y_pred = model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

#print(selected_features) #check to see if selected features are correct

Accuracy: 0.893
              precision    recall  f1-score   support

      Normal       0.94      0.88      0.91      1541
       Obese       0.91      0.90      0.91       789
  Overweight       0.85      0.88      0.87       991
 Underweight       0.85      0.94      0.89       679

    accuracy                           0.89      4000
   macro avg       0.89      0.90      0.89      4000
weighted avg       0.90      0.89      0.89      4000

Index(['Weight (kg)', 'Fat_Percentage', 'Water_Intake (liters)', 'BMI',
       'Calories', 'BMI_calc', 'protein_per_kg', 'cal_balance', 'lean_mass_kg',
       'Burns Calories (per 30 min)_bc'],
      dtype='object')
